# Ishihara Test Analysis using K-means Clustering

This notebook demonstrates how to extract numbers from Ishihara color blindness test images using K-means clustering with different color spaces.

## Setup

First, let's import the necessary packages and add the parent directory to the path so we can import our custom K-means implementation.

In [ ]:
import os
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Add the parent directory to the path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import our custom modules
from src.kmeans import KMeansClustering
from src.image_processing import preprocess_image, segment_image, extract_number
from src.color_spaces import visualize_color_channels, recommend_color_space
from src.visualization import plot_results, enhance_number_visibility

## Load an Ishihara Test Image

Let's load an Ishihara test image. You can replace this with your own image path.

In [ ]:
# Set the path to your Ishihara test image
image_path = "../../data/ishihara_74.png"

# Check if the image exists
if not os.path.exists(image_path):
    print(f"Error: Image not found at {image_path}")
    print("Please update the image_path variable to point to a valid image.")
else:
    # Load the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Display the image
    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.title("Original Ishihara Test Image")
    plt.axis("off")
    plt.show()

## Analyze Color Channels

Let's visualize the different color channels to understand which might be most effective for extracting the number.

In [ ]:
# Visualize RGB channels
fig, _, _, _ = visualize_color_channels(image, 'rgb')
plt.show()

# Visualize Lab channels
fig, _, _, _ = visualize_color_channels(image, 'lab')
plt.show()

# Visualize YCrCb channels
fig, _, _, _ = visualize_color_channels(image, 'ycrcb')
plt.show()

# Visualize HSV channels
fig, _, _, _ = visualize_color_channels(image, 'hsv')
plt.show()

## Get Color Space Recommendations

Let's use our recommendation system to identify the best color space and channel for this image.

In [ ]:
# Get recommendations
recommendations = recommend_color_space(image)

# Display recommendations
print("Recommended color spaces and channels for this image:")
print("------------------------------------------------------")
for i, rec in enumerate(recommendations):
    print(f"Recommendation {i+1}:")
    print(f"  Color space: {rec['color_space'].upper()}")
    print(f"  Channel: {rec['channel']}")
    print(f"  Explanation: {rec['explanation']}")
    print("")

## Process with Recommended Color Space

Now, let's use the top recommended color space to extract the number.

In [ ]:
# Get the top recommendation
top_rec = recommendations[0]
color_space = top_rec['color_space']
channel = top_rec['channel']
k = 2  # Usually 2 clusters works well
invert = False

print(f"Using {color_space.upper()} color space, channel {channel}, k={k}")

# Preprocess image
image_rgb, features, _ = preprocess_image(image_path, color_space, channel)

# Apply K-means clustering
kmeans = KMeansClustering(k=k, max_iterations=100)
kmeans.fit(features)

# Segment image
labels, segmented_image = segment_image(kmeans, features, image_rgb.shape, channel)

# Extract number
number_mask = extract_number(labels, k, invert)

# Display results
title = f"{color_space.upper()} color space, channel {channel}, k={k}"
plot_results(image_rgb, segmented_image, number_mask, title)
plt.show()

## Enhance Number Visibility

Let's enhance the visibility of the extracted number using contour detection.

In [ ]:
# Enhance number visibility
enhanced_number = enhance_number_visibility(number_mask)

# Display the enhanced number
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image_rgb)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(enhanced_number)
plt.title("Enhanced Number Visualization")
plt.axis("off")

plt.tight_layout()
plt.show()

## Try Different Color Spaces

Let's compare the results using different color spaces.

In [ ]:
# Define parameter sets to try
parameter_sets = [
    ('lab', 1, 2, False, 'Lab (a channel)'),
    ('ycrcb', 1, 2, False, 'YCrCb (Cr channel)'),
    ('hsv', 0, 3, False, 'HSV (Hue channel)'),
    ('rgb', 0, 2, False, 'RGB (Red channel)'),
    ('rgb', 1, 2, True, 'RGB (Green channel)'),
]

# Process with each parameter set
for color_space, channel, k, invert, title in parameter_sets:
    print(f"Processing with {title}...")
    
    # Preprocess image
    image_rgb, features, _ = preprocess_image(image_path, color_space, channel)
    
    # Apply K-means clustering
    kmeans = KMeansClustering(k=k, max_iterations=100)
    kmeans.fit(features)
    
    # Segment image
    labels, segmented_image = segment_image(kmeans, features, image_rgb.shape, channel)
    
    # Extract number
    number_mask = extract_number(labels, k, invert)
    
    # Display results
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image_rgb)
    plt.title("Original Image")
    plt.axis("off")
    
    plt.subplot(1, 3, 2)
    if len(segmented_image.shape) == 2:
        plt.imshow(segmented_image, cmap="gray")
    else:
        plt.imshow(segmented_image)
    plt.title(f"Segmented Image ({title})")
    plt.axis("off")
    
    plt.subplot(1, 3, 3)
    plt.imshow(number_mask, cmap="gray")
    plt.title("Extracted Number")
    plt.axis("off")
    
    plt.tight_layout()
    plt.show()

## K-means Implementation Analysis

Let's analyze our K-means implementation by visualizing how the centroids change during the clustering process.

In [ ]:
# Define a K-means implementation that records centroid history
class KMeansWithHistory(KMeansClustering):
    def __init__(self, k=3, max_iterations=100, random_state=42):
        super().__init__(k, max_iterations, random_state)
        self.centroid_history = []
    
    def fit(self, X):
        # Initialize centroids
        self.centroids = self.initialize_centroids(X)
        self.centroid_history = [self.centroids.copy()]
        
        # Run K-means algorithm
        for _ in range(self.max_iterations):
            # Compute distances
            distances = self.compute_distance(X, self.centroids)
            
            # Assign clusters
            labels = self.assign_clusters(distances)
            
            # Store old centroids for convergence check
            old_centroids = self.centroids.copy()
            
            # Update centroids
            self.centroids = self.update_centroids(X, labels)
            self.centroid_history.append(self.centroids.copy())
            
            # Check for convergence
            if self.has_converged(old_centroids, self.centroids):
                break
                
        return self

# Let's use the Lab color space, a-channel (which usually works well)
color_space = 'lab'
channel = 1
k = 2

# Preprocess image
image_rgb, features, _ = preprocess_image(image_path, color_space, channel)

# Apply K-means clustering with history tracking
kmeans = KMeansWithHistory(k=k, max_iterations=100)
kmeans.fit(features)

# Plot how centroids change over iterations
plt.figure(figsize=(10, 6))
colors = ['r', 'g', 'b', 'c', 'm']

for i in range(k):
    centroids_i = [ch[i][0] for ch in kmeans.centroid_history]
    plt.plot(range(len(centroids_i)), centroids_i, f'{colors[i%len(colors)]}-o', label=f'Centroid {i+1}')

plt.xlabel('Iteration')
plt.ylabel('Centroid Value')
plt.title(f'Centroid Convergence (Lab color space, a-channel)')
plt.legend()
plt.grid(True)
plt.show()

# Let's see after how many iterations the algorithm converged
print(f"K-means algorithm converged after {len(kmeans.centroid_history) - 1} iterations")

## Conclusion

In this notebook, we've demonstrated how to extract numbers from Ishihara color blindness test images using K-means clustering. We've seen that:

1. Different color spaces and channels have varying effectiveness for different Ishihara tests
2. The Lab color space's a-channel (red-green component) is often the most effective
3. Using just 2 clusters is usually sufficient for clear number extraction
4. Our K-means implementation converges efficiently to provide good segmentation

This approach can be applied to a wide variety of Ishihara test images to extract the hidden numbers.